# core

> the core functionalities of fastdebug

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

## make life easier with defaults  

In [ ]:
#| export
defaults = type('defaults', (object,), {'margin': 157 # align to the right by 157
                                        # 'eg': None, # examples
                                        # 'src': None, # official src
                                       }) 

## Execute strings

In [ ]:
eval?

Signature: eval(source, globals=None, locals=None, /)
Docstring:
Evaluate the given source in the context of globals and locals.

The source may be a string representing a Python expression
or a code object as returned by compile().
The globals must be a dictionary and locals can be any mapping,
defaulting to the current globals and locals.
If only globals is given, locals defaults to it.
Type:      builtin_function_or_method


In [ ]:
exec?

Signature: exec(source, globals=None, locals=None, /)
Docstring:
Execute the given source in the context of globals and locals.

The source may be a string representing one or more Python statements
or a code object as returned by compile().
The globals must be a dictionary and locals can be any mapping,
defaulting to the current globals and locals.
If only globals is given, locals defaults to it.
Type:      builtin_function_or_method


## make a colorful string

In [ ]:
#|export
class dbcolors:
    g = '\033[92m' #GREEN
    y = '\033[93m' #YELLOW
    r = '\033[91m' #RED
    reset = '\033[0m' #RESET COLOR

In [ ]:
#|export
def colorize(cmt, color:str=None):
    if color == "g":
        return dbcolors.g + cmt + dbcolors.reset
    elif color == "y":
        return dbcolors.y + cmt + dbcolors.reset
    elif color == "r":
        return dbcolors.r + cmt + dbcolors.reset
    else: 
        return cmt

In [ ]:
colorize("this is me", "r")

'\x1bthis is me\x1b'

In [ ]:
print(colorize("this is me", "r"))

this is me


## align text to the most right

In [ ]:
#| export
import re

In [ ]:
#| export
def strip_ansi(source):
    return re.sub(r'\033\[(\d|;)+?m', '', source)

In [ ]:
#| export
def alignright(blocks):
    lst = blocks.split('\n')
    maxlen = max(map(lambda l : len(strip_ansi(l)) , lst ))
    indent = defaults.margin - maxlen
    for l in lst:
        print(' '*indent + format(l))

In [ ]:
alignright("this is me")

                                                                                                                                                   this is me


## print out src code

In [ ]:
#| export
import inspect

### basic version

In [ ]:

def printsrc(src, srclines, cmt):
# print out the title
    print('\n')
    print('{:#^157}'.format(" srcline under investigation "))
    print('\n')


    # convert the source code of the function into a list of strings splitted by '\n'
    lst = inspect.getsource(src).split('\n')

    ccount = 0
    for l in lst: 
        if bool(l) and l.strip() in srclines: # print out the srcline under investigation
            print('{:=<157}'.format(l))
            ccount = ccount + 1

            if bool(cmt): # print out comment at the end of the srclines under investigation
                numsrclines = len(srclines.split("\n"))
                if ccount == numsrclines: 
                    colcmt = colorize(cmt, "r") # colorize the comment
                    alignright(colcmt) # put the comment to the most right

        else: 
            print('{:<157}'.format(l)) # print out the rest of source code

In [ ]:
def foo():     
    a = 1 + 1
    b = a + 1
    pass

In [ ]:
printsrc(foo, "a = 1 + 1", "this is comment")



################################################################ srcline under investigation ################################################################


def foo():                                                                                                                                                   
    a = 1 + 1================================================================================================================================================
                                                                                                                                              this is comment
    b = a + 1                                                                                                                                                
    pass                                                                                                                                                     
                                                

In [ ]:
printsrc(foo, "    a = 1 + 1\n    b = a + 1", "this is comment")



################################################################ srcline under investigation ################################################################


def foo():                                                                                                                                                   
    a = 1 + 1================================================================================================================================================
    b = a + 1================================================================================================================================================
                                                                                                                                              this is comment
    pass                                                                                                                                                     
                                                

### print src with specific number of lines

In [ ]:
#| export
def printsrc(src, srclines, cmt, lines:int=5):
# print out the title
    print('\n')
    print('{:#^157}'.format(" srcline under investigation "))
    print('\n')


    # convert the source code of the function into a list of strings splitted by '\n'
    lst = inspect.getsource(src).split('\n')
    
    # find the idx of the first srcline under investigation in src code
    

    ccount = 0
    for l in lst: 
        if bool(l) and l.strip() in srclines: # print out the srcline under investigation with # as padding
            print('{:=<157}'.format(l))
            ccount = ccount + 1

            if bool(cmt): # print out comment at the end of the srclines under investigation
                numsrclines = len(srclines.split("\n"))
                if ccount == numsrclines: 
                    colcmt = colorize(cmt, "r") # colorize the comment
                    alignright(colcmt) # put the comment to the most right

        else: 
            print('{:<157}'.format(l)) # print out the rest of source code to the most left

    

## dbprint on expression

### basic version

In [ ]:
#| export
def dbprint(src, # the src func name, e.g., foo
            srclines:str, # the srclines under investigation
            cmt:str, # comment
            *code,   # a number of codes to run, each code is in str, e.g., "a + b", "c = a - b"
            **env
           ):  # a number of stuff needed to run the code, e.g. var1 = var1, func1 = func1
    "debug a srcline with one or more expressions with src printed."
    
    # print out src code: the basic version
    printsrc(src, srclines, cmt)
    
    for c in code:
    # print(f"{c} => {c} : {eval(c, globals().update(env))}") 
        output = f"{c} => {c} : {eval(c, globals().update(env))}"
        print('{:>157}'.format(output))   

In [ ]:
def foo():     
    a = 1 + 1
    b = a + 1
    pass

In [ ]:
def foo(): 
    dbprint(foo, "    a = 1 + 1", "this is a test", "1+2", "str(1+2)")
    a = 1 + 1
    pass

In [ ]:
foo()



################################################################ srcline under investigation ################################################################


def foo():                                                                                                                                                   
    dbprint(foo, "    a = 1 + 1", "this is a test", "1+2", "str(1+2)")                                                                                       
    a = 1 + 1================================================================================================================================================
                                                                                                                                               this is a test
    pass                                                                                                                                                     
                                                

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

#|hide
## Export

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()

#|hide
## Send to Obsidian

In [ ]:
#| hide
!jupytext --to md /Users/Natsume/Documents/fastdebug/00_core.ipynb
!mv /Users/Natsume/Documents/fastdebug/00_core.md \
/Users/Natsume/Documents/divefastai/Debuggable/jupytext/

[jupytext] Reading /Users/Natsume/Documents/fastdebug/00_core.ipynb in format ipynb
[jupytext] Writing /Users/Natsume/Documents/fastdebug/00_core.md


In [ ]:
#| hide
!jupyter nbconvert --config /Users/Natsume/Documents/mynbcfg.py --to markdown \
--output-dir /Users/Natsume/Documents/divefastai/Debuggable/nbconvert

[NbConvertApp] WARNING | pattern '/Users/Natsume/Documents/fastdebug/fastcore/*.ipynb' matched no files
[NbConvertApp] WARNING | pattern '/Users/Natsume/Documents/fastdebug/fastcore/meta/*.ipynb' matched no files
[NbConvertApp] WARNING | pattern '/Users/Natsume/Documents/fastdebug/fastcore/findings/*.ipynb' matched no files
[NbConvertApp] Converting notebook /Users/Natsume/Documents/fastdebug/index.ipynb to markdown
[NbConvertApp] Writing 306 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/index.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/fastdebug/FooGetSigInit.ipynb to markdown
[NbConvertApp] Writing 39305 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/FooGetSigInit.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/fastdebug/00_core.ipynb to markdown
[NbConvertApp] Writing 11849 bytes to /Users/Natsume/Documents/divefastai/Debuggable/nbconvert/00_core.md
[NbConvertApp] Converting notebook /Users/Natsume/Documents/fa